<!-- Autoheader begin -->
<hr/>
<div id="navtitle_2_3_jl" style="text-align:center; font-size:16px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_2_lambda.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_1_3_chirp.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_1_3_chirp.ipynb" style="font-size:13px">I.3 Interaction of a Two-Level-System with a Chirped Laser Pulse</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_1_TLS.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method and GRAPE</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a><br>
        <a href="jl_exercise_3_3_chiral.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_2_3_chiral.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Parameter Optimization of Three-Wave Mixing in a Three-Level System

This notebook performs optimization of three-wave mixing in a chiral three-level system.
In this notebook you will learn how to use gradient-free parameter optimization via the `NLopt` package (via `Optimization.jl`) for the purpose of driving two three-level systems, representing the two mirror images (the so-called enantiomers) of a chiral molecules, such that they end up in distinct final states. This allows for the discrimination of enantiomers which is a central task in applications involving chiral molecules. As a bonus, we also show how gradient information via a technique called automatic differentiation can be used to improve the performance of the optimization.

## Model

<img src="../figures/3-level_mod.svg" alt="Drawing" style="width: 800px;"/>

The illustration above shows two a three-level model for a chiral molecules with the left and right side corresponding to the $+$(left), respectively $-$(right), enantiomer of the molecule.
The three levels are connected with each other by an electric dipole transition. The only difference between the two enantiomers lies in the sign of the $\mu_{c}$ component of the dipole transition moment. Our goal is to obtain 'enantioselectivity' which means that a sequence of microwave pulses applied to the two enantiomers starting in the same initial state will lead to two perfectly distinct final states.

## Setup

We start with importing the necessary packages, among which are
* `QuantumPropagators`: Allows to set up time dependent Hamiltonians and propagate them.
* `OrdinaryDiffEq`: This is the backend used for `QuantumPropagators.propagate`.
* `Optimization` contains routines for optimization, with `OptimizationNLopt` providing the connection to the `NLOpt` backend package.
* `ComponentArrays` provides `ComponentVector` which allows for an elegant definition of vectors of control parameters which can be accessed both by index and by name. The `@unpack` macro from `UnPack.jl` can be used to unpack the parameters from the vector into individual variables.

In [ ]:
using QuantumPropagators: hamiltonian, propagate
using OrdinaryDiffEq
using Optimization, OptimizationNLopt
using ComponentArrays: ComponentVector
using UnPack: @unpack

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

In [ ]:
# Some utilities for showing hints and solutions
include(joinpath("utils", "exercise_2_chiral.jl"));

To make our code slightly easier to read, we define a constant 𝕚 for the imaginary unit:

In [ ]:
const 𝕚 = 1im;  # "𝕚" can be typed by \bbi<tab>

## Hamiltonian
We begin by defining the function `total_enantiomer_ham`. This function returns the Hamiltonian (in the rotating frame) for the two enantiomers, i.e.,

$$
\hat{H}_{I}^{(\pm)}(t) = \sum_{i=1}^{3} \hat{H}_{i}^{(\pm)}(t)
$$

with

$$
\hat{H}_{1}^{(\pm)}(t)
=
- E_{1}(t) \mu_{b}^{(\pm)}
\begin{pmatrix}
  0 & e^{i \phi_{1}} & 0 \\
  e^{- i \phi_{1}} & 0 & 0 \\
  0 & 0 & 0 \\
\end{pmatrix},
\\
\hat{H}_{2}^{(\pm)}(t)
=
- E_{2}(t) \mu_{a}^{(\pm)}
\begin{pmatrix}
  0 & 0 & 0 \\
  0 & 0 & e^{i \phi_{2}} \\
  0 & e^{- i \phi_{2}} & 0 \\
\end{pmatrix},
\\
\hat{H}_{3}^{(\pm)}(t)
=
- E_{3}(t) \mu_{c}^{(\pm)}
\begin{pmatrix}
  0 & 0 & e^{i \phi_{3}} \\
  0 & 0 & 0 \\
  e^{- i \phi_{3}} & 0 & 0 \\
\end{pmatrix}
$$

and where

$$
E_{i}(t)
=
\frac{E_{i,0}}{2}
\left[\tanh(a (t - t_{i,1})) - \tanh(a (t - t_{i,2}))\right]
$$

for $i \in \left\{1,2,3\right\}$ are the envelopes of the pulses with frequencies $\omega, \delta \omega$ and $\omega + \delta \omega$.

In [ ]:
E(t; E₀, t₁, t₂, a) =  (E₀/2) * (tanh(a*(t-t₁)) - tanh(a*(t-t₂)));

The `total_enantiomer_ham` function gets a set of control `parameters`, which altogether specify the Hamiltonian and the control fields in their entirety. We use here a "structured" [`ComponentVector`](https://jonniedie.github.io/ComponentArrays.jl/stable/api/#ComponentArrays.ComponentVector) with three named sub-vectors:

* `ΔT`: A vector of the three durations $\Delta t_{i} = t_{i,2} - t_{i,1}$ of each of the three fields $E_{i0}(t)$. Field $i$ is assumed to start when field $i-1$ ends. The first field starts at $t_{1,1}=0$.
* `ϕ`: A vector of the three real phases $\phi_{i}$ for each field.
* `E₀`: A vector of the three real amplitudes $E_{i0}$ for each field.

These subsets are combined as follows

In [ ]:
parameters = ComponentVector(ΔT=[0.2, 0.4, 0.3], ϕ=[π, π, π], E₀=[0.45, 0.4, 0.5]);

The `parameters` array acts as an array with 9 elements that we can access individually:

In [ ]:
parameters[1:9]

Alternatively, we can also access the sub-vectors by their name:

In [ ]:
parameters.ΔT

or [unpack](https://github.com/mauro3/UnPack.jl) the entire array (or part of the array) into variables:

In [ ]:
@unpack ΔT, E₀ = parameters;

The `total_enantiomer_ham` function also uses the following keyword arguments:

* `sign`: The string `+` or `-` specifies which Hamiltonian, i.e., $H_{I}^{(+)}(t)$ or $H_{I}^{(-)}(t)$, is retuned.
* `a`: This is a parameter which controls how smooth each field is turned on and off. The larger `a` becomes, the more the field shapes $E_{i0}(t)$ resemble a rectangle.

In [ ]:
function total_enantiomer_ham(parameters; sign, a)

    @unpack ϕ, ΔT, E₀ = parameters
    ϕ₁, ϕ₂, ϕ₃ = ϕ[1:3]
    μ = (sign == "-" ? -1 : 1)

    H₁ = μ * [
             0     exp(𝕚*ϕ₁)  0
        exp(-𝕚*ϕ₁)     0      0
             0         0      0
    ]

    H₂ = μ * [
        0       0          0
        0       0      exp(𝕚*ϕ₂)
        0  exp(-𝕚*ϕ₂)      0
    ]

    H₃ = μ * [
              0      0  exp(𝕚*ϕ₃)
              0      0      0
         exp(-𝕚*ϕ₃)  0      0
    ]

    # times where pulses end
    T₁ = sum(ΔT[1:1])
    T₂ = sum(ΔT[1:2])
    T₃ = sum(ΔT[1:3])

    return hamiltonian(
        (H₁, t -> E(t; E₀=E₀[1], t₁=0.0, t₂=T₁, a)),
        (H₂, t -> E(t; E₀=E₀[2], t₁=T₁, t₂=T₂, a)),
        (H₃, t -> E(t; E₀=E₀[3], t₁=T₂, t₂=T₃, a)),
    )
end

In [ ]:
H₊ = total_enantiomer_ham(parameters; sign="+", a=100);

We define the initial state as

$$
\Psi_{\pm}(0)
=
\begin{pmatrix}
  1 \\ 0 \\ 0
\end{pmatrix}.
$$

In [ ]:
# the initial state consists of three levels with population initially in the ground state
Ψ₀ = ComplexF64[1, 0, 0];

Our time grid is obtained by dividing $$t \in \left[0,1\right]$$ into 100 equal intervals (remember that 100 intervals implies 101 points of the time grid).

In [ ]:
tlist = collect(range(0, 1; length=101));

## Problem 0 - Pulse parameterisation

We begin by familiarizing ourselves with the pulse parameterization. In this notebook the pulses are formed by the difference between two hyperbolic tangent functions.
`E0` controls the pulse amplitude, `a` controls how rectangular the pulse appears and `t_start` and `t_stop` determine when the pulse starts and ends.
Try changing the arguments of `plot_parameterised_pulse`, such that the two curves match (an exact fit is difficult, it is sufficient to aim for a value of the calculated mismatch which is below one).

In [ ]:
function plot_parameterised_pulse(tlist; E₀, a, t₁, t₂)
    pulse = t -> E(t; E₀, a, t₁, t₂)
    target_pulse = t -> 20 * exp(-20 * (t - 0.5)^2)
    mismatch = sum(abs.(pulse.(tlist) .- target_pulse.(tlist))) / length(tlist)
    plot(tlist, pulse , label="your pulse", color=(mismatch < 1 ? "green" : "blue"))
    plot!(tlist, target_pulse; label="target pulse", color="orange")
    annotate!(
        0, 20,
        ("mismatch: $(round(mismatch; digits=3))", 10, :left)
    )
end

plot_parameterised_pulse(tlist; E₀=3.14, a=500, t₁=0.15, t₂=0.85)

In [ ]:
# problem_0.hint

In [ ]:
# problem_0.solution

**Bonus**: After you go through the "Problem 1" below, you can come back here and use the `Optimization` package to determine even better parameters.

In [ ]:
#bonus.hint

In [ ]:
#bonus.solution

## Initialise optimization

Now we can turn towards optimization. To this end, we use the optimization methods provided by the [`NLOpt` package](https://nlopt.readthedocs.io/en/latest/). This package has a [Julia wrapper](https://github.com/JuliaOpt/NLopt.jl) which we could use directly, but there is also a package [Optimization.jl](https://docs.sciml.ai/Optimization/stable/) that provides a common API for defining and solving optimization problems not just via `NLOpt` but also a dozen other optimization toolboxes. For simplicity, we use the well-known Nelder-Mead method. Note, however, that `NLOpt` allows for a wide range of different methods as do the other toolboxes wrapped by Optimization.jl.

The Optimization.jl interface requires to provide a `loss` function, i.e. the optimization functional, which takes a vector `x` of real-valued parameters as well as an object `constants` containing static parameters. This could be another vector, but a [`NamedTuple`](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) is often cleaner. Note that in a different setup, the function to be optimized could be a more general [`OptimizationFunction`](https://docs.sciml.ai/Optimization/stable/API/optimization_function/) object which also contains information on how to calculate gradients. See the [Optimization.jl manual](https://docs.sciml.ai/Optimization/stable/API/optimization_problem/) for details. Since we only study gradient-free optimization in this notebook, we will not need this here.

Our `loss` function takes the list `x` containing our optimization parameters, i.e., all pulse durations, all phases $\phi_{i}$ and all amplitudes $E_{i,0}$ on inputand returns the error of the enantioselectivity protocol introduced above. Specifically, `loss` returns zero if and only if the dynamics obtained from the parameters in the set `x` transfers the initial state $\Psi(0)$ perfectly into the target state, i.e.,


$$
\Psi_{+}(0)
\longrightarrow
\Psi_{+}\left(T\right) =
\begin{pmatrix}
  1 \\ 0 \\ 0
\end{pmatrix}
$$

for enantiomer `+` and

$$
\Psi_{-}(0)
\longrightarrow
\Psi_{-}\left(T\right) =
\begin{pmatrix}
  0 \\ 0 \\ 1
\end{pmatrix}
$$

for enantiomer `-`.

In [ ]:
function loss(x, constants)

    # The optimizer will usually pass in `x` as a standard vector,
    # so we have to repack it into a `ComponentVector` for
    # `total_enantiomer_ham`
    parameters = ComponentVector(ΔT=x[1:3], ϕ=x[4:6], E₀=x[7:9])

    H₊ = total_enantiomer_ham(parameters; sign="+", a=constants.a)
    H₋ = total_enantiomer_ham(parameters; sign="-", a=constants.a)

    Ψ₀ = ComplexF64[1, 0, 0];
    tlist = collect(range(0, constants.T; length=constants.nt));

    Ψ₊ = propagate(Ψ₀, H₊, tlist; method=OrdinaryDiffEq)
    Ψ₋ = propagate(Ψ₀, H₋, tlist; method=OrdinaryDiffEq)

    fid = (abs2(Ψ₊[1]) + abs2(Ψ₋[3])) / 2
    return 1 - fid

end

## Problem 1 - Run optimization

Next, we can run the actual optimization. It requires us to define lower and upper bounds for all parameters that should be optimized. In our case, we choose

$$
0 \leq t_{i,2} \leq T=1,
\qquad
0 \leq \phi_{i} \leq 2 \pi,
\qquad
0 \leq E_{i,0} \leq 10.
$$

Note that we need to provide guess values for all nine parameters. The order that the parameters should appear in is `ΔT₁`, `ΔT₂`, `ΔT₃`, `ϕ₁`, `ϕ₂`, `ϕ₃`, `E₀₁`, `E₀₂`, `E₀₃`.

The choice of the guess values will often have an appreciable impact on the general success of the optimized solution and can even affect its form since many optimization problems allow for many different solutions.
Your task is now to fill in the upper optimization bounds and to try different
guesses to evaluate their impact on the optimization.

In [ ]:
guess = ComponentVector(
    ΔT=[#= insert values =#],
    ϕ=[#= insert values =#],
    E₀=[#= insert values =#]
);

prob = OptimizationProblem(
    loss,
    guess,
    (a=1000.0, T=1, nt=100);  # this is a NamedTuple
    lb=zeros(length(guess)),
    ub=ComponentVector(
        ΔT=[#= insert values =#],
        ϕ=[#= insert values =#],
        E₀=[#= insert values =#]
    ),
    stopval=(1-0.99),
);

In [ ]:
# problem_1.hint

In [ ]:
# problem_1.solution

We can check the performance of the guess pulse:

In [ ]:
loss(guess, (a=1000.0, T=tlist[end], nt=length(tlist)))

In the optimization, we'll want to keep track of the fidelity after each iteration. To this end, we define a ["callback" function](https://docs.sciml.ai/Optimization/stable/API/solve/#CommonSolve.solve-Tuple%7BOptimizationProblem,%20Any%7D) that the optimizer will run after each step.

In [ ]:
obtained_fidelities = Float64[];  # for keeping track of the fidelity in each iteration

function callback(state, loss_val)
    global obtained_fidelities
    fid = 1 - loss_val
    push!(obtained_fidelities, fid)
    print("Iteration: $(length(obtained_fidelities)), current fidelity $(round(fid; digits=4))\r")
    return false
end

Lastly, we call `Optimization.solve` to run the optimization:

In [ ]:
obtained_fidelities = Float64[];
res = Optimization.solve(prob, NLopt.LN_NELDERMEAD(); maxiters=500, callback)

In [ ]:
plot(obtained_fidelities; marker=:cross, label="", xlabel="optimization iteration", ylabel="fidelity")

## Analyze optimization results
After optimiziation we can verify the optimization result by plotting the pulses
as well as the resulting dynamics.
To this end, we define the Hamiltonians of the two enantiomers given the optimized parameters.

In [ ]:
opt_params = convert(typeof(guess), res.u)

H₊opt = total_enantiomer_ham(opt_params; sign="+", a=1000.0);
H₋opt = total_enantiomer_ham(opt_params; sign="-", a=1000.0);

In order to visualize the optimized pulses, we plot them in the following. We use

In [ ]:
using QuantumPropagators.Controls: get_controls

to extract the functions with the optimized parameters from the optimal Hamiltonians.

In [ ]:
function plot_pulse(H, tlist; kwargs...)
    fig = plot(; xlabel="time", ylabel="amplitude", kwargs...)
    sub = ["₁", "₂", "₃"]
    for (i, E) in enumerate(get_controls(H))
        plot!(fig, tlist, E; label="E$(sub[i])")
    end
    return fig
end

plot_pulse(H₊opt, range(0, 1; length=500); legend=:right)

Finally, we solve the dynamics of the two enantiomers (now using the optimized parameters) and plot their population dynamics.

In [ ]:
function propagate_system(Htot, title; tlist=tlist)
    Ψ₀ = ComplexF64[1, 0, 0];
    states = propagate(Ψ₀, Htot, tlist; method=OrdinaryDiffEq, storage=true)
    pops = abs2.(states)
    plot(; title, xlabel="time", ylabel="population", legend=:top)
    plot!(tlist, pops[1,:]; label="|c₁|²")
    plot!(tlist, pops[2,:]; label="|c₂|²")
    plot!(tlist, pops[3,:]; label="|c₃|²")
end

In [ ]:
propagate_system(H₊opt, "enantiomer +")

In [ ]:
propagate_system(H₋opt, "enantiomer -")

In the plots above you see how the population evolves under the influence of our parameterised pulses. If the optimization was successful, the populations of the two enantiomers at the final time $T$ should be entirely in the $\Psi_{+}(T)$ (first level) and $\Psi_{-}(T)$ (third level) target states respectively.

## Advanced: Gradient-Based Optimization with Automatic Differentiation

In many cases, the convergence of optimizations can be improved by taking into account information about the gradient of the loss functional. In fact, the gradient-free optimization method we have used above will only perform in a sensible amount of time for a relatively small number of control parameters. Beyond that, the use of gradient-based approaches is imperative.

Unfortunately, evaluating the gradient of the `loss` function with respect to the parameters is highly non-straightforward. A potential way out is to use a technique called "automatic differentiation", where the computer tracks all computational steps and uses a stupendous application of the chain rule to obtain the gradient.

There are several frameworks for automatic differentiation in Julia, the most established one probably being [`Zygote`](https://github.com/FluxML/Zygote.jl).

In [ ]:
using Zygote

It can be used together with the `OrdinaryDiffEq` solver that we have already used throughout our notebooks as the numerical backend for simulating the time evolution.

In [ ]:
using SciMLSensitivity

The above command activates these capabilities. However, this also means that we have to use `OrdinaryDiffEq` *directly*, not via `QuantumPropagators.propagator`. There is an even more severe restrictions that come with using automatic differentiation via Zygote: It does not support in-place linear algebra operators. In each time step, we must construct the Hamiltonian matrix from the time-dependent `Generator` returned by `total_enantiomer_ham`.

Lastly, accurate gradients require high precision in the ODE solver, so we have to lower the default `reltol` and `abstol` values. It also improves the convergence to adjust the lower bounds of control parameters. In particular, hitting the lower bound `ΔT₁=0.0` is a pathological case that will confuse the optimizer. Hence, it is best to explicitly keep all `ΔT` and `E₀` values higher than zero.

The framework discussed above is set up as follows:

In [ ]:
using QuantumPropagators.Controls: evaluate

function f₊(Ψ, p, t)
    params = ComponentVector(ΔT=p[1:3], ϕ=p[4:6], E₀=p[7:9])
    H = total_enantiomer_ham(params; sign="+", a=1000.0)
    op = evaluate(H, t)  # Evaluate H(t) as a matrix
    return -1im * op * Ψ
end

function f₋(Ψ, p, t)
    params = ComponentVector(ΔT=p[1:3], ϕ=p[4:6], E₀=p[7:9])
    H = total_enantiomer_ham(params; sign="-", a=1000.0)
    op = evaluate(H, t)
    return -1im * op * Ψ
end

Or, more directly:

In [ ]:
function f(Ψ, p, t; sign="+", a=1000.0)
    ΔT₁, ΔT₂, ΔT₃, ϕ₁, ϕ₂, ϕ₃, E₀₁, E₀₂, E₀₃ = p
    T₁ = ΔT₁
    T₂ = ΔT₁ + ΔT₂
    T₃ = ΔT₁ + ΔT₂ + ΔT₃
    μ = (sign == "-" ? -1 : 1)
    E₁ = E(t; E₀=E₀₁, t₁=0.0, t₂=T₁, a)
    E₂ = E(t; E₀=E₀₂, t₁=T₁, t₂=T₂, a)
    E₃ = E(t; E₀=E₀₃, t₁=T₂, t₂=T₃, a)
    F = (-𝕚 * µ) * [  # -𝕚 * H  (RHS of Schrödinger Eq. rewritten as ODE)
              0.0           E₁ * exp(𝕚 * ϕ₁)   E₃ * exp(𝕚 * ϕ₃)
        E₁ * exp(-𝕚 * ϕ₁)         0.0          E₂ * exp(𝕚 * ϕ₂)
        E₃ * exp(-𝕚 * ϕ₃)   E₂ * exp(-𝕚 * ϕ₂)         0.0
    ]
    return F * Ψ
end

f₊(Ψ, p, t) = f(Ψ, p, t; sign="+", a=1000.0);
f₋(Ψ, p, t) = f(Ψ, p, t; sign="-", a=1000.0);

In [ ]:
# XXX
guess = ComponentVector(
    ΔT=[0.2, 0.4, 0.3],
    ϕ=[π, π, π],
    E₀=[5.0, 5.0, 5.0]
);

In [ ]:
function loss_zygote(x)

    Ψ₀ = ComplexF64[1, 0, 0];
    tspan = (0.0, 1.0)

    prob₊ = ODEProblem(f₊, Ψ₀, tspan, x)
    prob₋ = ODEProblem(f₋, Ψ₀, tspan, x)

    Ψ₊ = OrdinaryDiffEq.solve(prob₊, DP5(), reltol = 1e-9, abstol = 1e-7, verbose=false).u[end]
    Ψ₋ = OrdinaryDiffEq.solve(prob₋, DP5(), reltol = 1e-9, abstol = 1e-7, verbose=false).u[end]

    fid = (abs2(Ψ₊[1]) + abs2(Ψ₋[3])) / 2
    return 1 - fid

end

loss_zygote(guess)

We can look at the gradient of the guess:

In [ ]:
grad = loss_zygote';
grad(guess)

In [ ]:
prob_zygote = OptimizationProblem(
    OptimizationFunction((x, _)->loss_zygote(x), AutoZygote()),
    guess,
    nothing;
    lb=ComponentVector(
        ΔT=[0.1, 0.1, 0.1],
        ϕ=[0.0, 0.0, 0.0],
        E₀=[1.0, 1.0, 1.0]
    ),
    ub=ComponentVector(
        ΔT=[1.0, 1.0, 1.0],
        ϕ=[2π, 2π, 2π],
        E₀=[10.0, 10.0, 10.0]
    ),
    stopval=(1-0.99),
);

And now run the optimization using the gradient-based LBFGS quasi-Newton optimization method with the Zygote-derived gradients.

In [ ]:
obtained_fidelities = Float64[];
res_zygote = Optimization.solve(prob_zygote, NLopt.LD_LBFGS(), maxiters=500, callback=callback)

In [ ]:
plot(obtained_fidelities; marker=:cross, label="", xlabel="optimization iteration", ylabel="fidelity")

In [ ]:
opt_params = convert(typeof(guess), res_zygote.u)

H₊opt = total_enantiomer_ham(opt_params; sign="+", a=1000.0);
H₋opt = total_enantiomer_ham(opt_params; sign="-", a=1000.0);

In [ ]:
plot_pulse(H₊opt, range(0, 1; length=500); legend=:right)

In [ ]:
propagate_system(H₊opt, "enantiomer +")

In [ ]:
propagate_system(H₋opt, "enantiomer -")

## Next steps

As alluded to already in the "Advanced" section above, in many cases gradient-based optimization is a powerful alterantive to parameter optimization. A thorough, introductory notebook for this topic can be found in [Exercise III.2](jl_exercise_3_2_lambda.ipynb) where the example of the STIRAP protocol is discussed. Alternatively, if you would like to stick with chiral molecules and three-wave-mixing, have a look at [Exercise III.3](jl_exercise_3_3_chiral.ipynb), in which gradient-based opimization is employed for the problem you studied in this notebook.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_2_3_jl)
<!-- Autofooter end -->